# TCN Classification with CR Dataset
<hr>

We will build a text classification model using TCN model on the Customer Reviews Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 


## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [1]:
# Import the library in your Jupyter notebook
import jovian

In [4]:
jovian.commit()

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Updating notebook "diardanoraihan/tcn-cr" on https://jovian.ai/
[jovian] Uploading notebook..
[jovian] Capturing environment..
[jovian] Committed successfully! https://jovian.ai/diardanoraihan/tcn-cr


'https://jovian.ai/diardanoraihan/tcn-cr'

In [2]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/CR/CR.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(3775, 3)


,sentence,label,split
0,weaknesses are minor the feel and layout of th...,0,train
1,many of our disney movies do n 't play on this...,0,train
2,player has a problem with dual layer dvd 's su...,0,train
3,i know the saying is you get what you pay for ...,0,train
4,will never purchase apex again .,0,train
...,...,...,...
3770,"so far , the anti spam feature seems to be ver...",1,train
3771,i downloaded a trial version of computer assoc...,1,train
3772,i did not have any of the installation problem...,1,train
3773,their products have been great and have saved ...,1,train


In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3775 entries, 0 to 3774
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  3775 non-null   object
 1   label     3775 non-null   int32 
 2   split     3775 non-null   object
dtypes: int32(1), object(2)
memory usage: 73.9+ KB


In [5]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,1368,1368
1,2407,2407


In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [8]:
sentences[0]

"weaknesses are minor the feel and layout of the remote control are only so so . it does n 't show the complete file names of mp3s with really long names . you must cycle through every zoom setting ( 2x , 3x , 4x , 1 2x , etc . ) before getting back to normal size sorry if i 'm just ignorant of a way to get back to 1x quickly ."

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [9]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  will never purchase apex again .
Into a sequence of int: [72, 194, 285, 207, 286]
Into a padded sequence: [ 72 194 285 207 286   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0]


In [11]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
and 3
i 4
it 5
to 6
a 7
is 8
of 9
this 10
5336


# Model 1: Embedding Random
<hr>

## TCN Model

Now, we will build Temporal Convolutional Network (CNN) models to classify encoded documents as either positive or negative.

The model takes inspiration from https://github.com/philipperemy/keras-tcn and https://www.kaggle.com/christofhenkel/temporal-convolutional-network

__Arguments__
`TCN(nb_filters=64, kernel_size=2, nb_stacks=1, dilations=[1, 2, 4, 8, 16, 32], padding='causal', use_skip_connections=False, dropout_rate=0.0, return_sequences=True, activation='relu', kernel_initializer='he_normal', use_batch_norm=False, **kwargs)`

- `nb_filters`: Integer. The number of filters to use in the convolutional layers. Would be similar to units for LSTM.
- `kernel_size`: Integer. The size of the kernel to use in each convolutional layer.
- `dilations`: List. A dilation list. Example is: [1, 2, 4, 8, 16, 32, 64].
- `nb_stacks`: Integer. The number of stacks of residual blocks to use.
- `padding`: String. The padding to use in the convolutions. 'causal' for a causal network (as in the original implementation) and - `'same' for a non-causal network.
- `use_skip_connections`: Boolean. If we want to add skip connections from input to each residual block.
- `return_sequences`: Boolean. Whether to return the last output in the output sequence, or the full sequence.
- `dropout_rate`: Float between 0 and 1. Fraction of the input units to drop.
- `activation`: The activation used in the residual blocks o = activation(x + F(x)).
- `kernel_initializer`: Initializer for the kernel weights matrix (Conv1D).
- `use_batch_norm`: Whether to use batch normalization in the residual layers or not.
- `kwargs`: Any other arguments for configuring parent class Layer. For example "name=str", Name of the model. Use unique names when using multiple TCN.

Now, we will define our TCN model as follows:
- One TCN layer with 100 filters, kernel size 1-6, and relu and tanh activation function;
- Dropout size = 0.5;
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: binary cross-entropy (suited for binary classification problem)

**Note**: 
- The whole purpose of dropout layers is to tackle the problem of over-fitting and to introduce generalization to the model. Hence it is advisable to keep dropout parameter near 0.5 in hidden layers. 
- https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/

In [44]:
from tcn import TCN, tcn_full_summary
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model

def define_model(kernel_size = 3, activation='relu', input_dim = None, output_dim=300, max_length = None ):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length)(inp)
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [45]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_15 (Embedding)        (None, 100, 300)     1532400     input_3[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_3 (SpatialDro (None, 100, 300)     0           embedding_15[0][0]               
__________________________________________________________________________________________________
tnc1 (TCN)                      (None, 100, 128)     279936      spatial_dropout1d_3[0][0]        
____________________________________________________________________________________________

In [43]:
# tcn_full_summary(model_0)

In [103]:
# class myCallback(tf.keras.callbacks.Callback):
#     # Overide the method on_epoch_end() for our benefit
#     def on_epoch_end(self, epoch, logs={}):
#         if (logs.get('accuracy') > 0.93):
#             print("\nReached 93% accuracy so cancelling training!")
#             self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [104]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu', 'tanh']
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp = 0

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1

            # Define the input shape
            model = define_model(kernel_size, activation, input_dim=vocab_size, max_length=max_len)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record = record.append(temp, ignore_index=True)
        print()
        print(record)
        print()

C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------
Epoch 1/100
68/68 [==============================] - 13s 147ms/step - loss: 0.6550 - accuracy: 0.6398 - val_loss: 0.6037 - val_accuracy: 0.6032
Epoch 2/100
68/68 [==============================] - 9s 126ms/step - loss: 0.5296 - accuracy: 0.6992 - val_loss: 0.4230 - val_accuracy: 0.7778
Epoch 3/100
68/68 [==============================] - 9s 126ms/step - loss: 0.3123 - accuracy: 0.8805 - val_loss: 0.4155 - val_accuracy: 0.8042
Epoch 4/100
68/68 [==============================] - 9s 127ms/step - loss: 0.1468 - accuracy: 0.9485 - val_loss: 0.5420 - val_accuracy: 0.8069
Epoch 5/100
68/68 [==============================] - 9s 126ms/step - loss: 0.0652 - accuracy: 0.9786 - val_loss: 0.6025 - val_accuracy: 0.8175
Epoch 6/100
68/68 [==============================] - 9s 126ms/step - loss: 0.0390 - accuracy: 0.9848 - val_loss: 0.9324 - val_accuracy: 0.8122
Epoch 7/1

Epoch 8/100
68/68 [==============================] - 9s 137ms/step - loss: 0.0498 - accuracy: 0.9818 - val_loss: 1.0927 - val_accuracy: 0.7937
Epoch 9/100
68/68 [==============================] - 9s 136ms/step - loss: 0.0130 - accuracy: 0.9963 - val_loss: 1.2246 - val_accuracy: 0.8069
Epoch 10/100
68/68 [==============================] - 9s 136ms/step - loss: 0.0093 - accuracy: 0.9982 - val_loss: 1.2347 - val_accuracy: 0.7831
Epoch 11/100
68/68 [==============================] - 9s 138ms/step - loss: 0.0531 - accuracy: 0.9844 - val_loss: 1.1576 - val_accuracy: 0.7857
Epoch 12/100
68/68 [==============================] - 10s 149ms/step - loss: 0.0057 - accuracy: 0.9992 - val_loss: 1.4119 - val_accuracy: 0.7831
Epoch 13/100
68/68 [==============================] - 10s 143ms/step - loss: 0.0106 - accuracy: 0.9954 - val_loss: 1.1926 - val_accuracy: 0.7672
Epoch 14/100
68/68 [==============================] - 10s 140ms/step - loss: 0.0167 - accuracy: 0.9938 - val_loss: 1.2020 - val_accuracy

68/68 [==============================] - 9s 139ms/step - loss: 0.0292 - accuracy: 0.9889 - val_loss: 1.0207 - val_accuracy: 0.7692
Epoch 8/100
68/68 [==============================] - 9s 139ms/step - loss: 0.0163 - accuracy: 0.9950 - val_loss: 1.2392 - val_accuracy: 0.7825
Epoch 9/100
68/68 [==============================] - 9s 139ms/step - loss: 0.0286 - accuracy: 0.9914 - val_loss: 1.3920 - val_accuracy: 0.7772
Epoch 10/100
68/68 [==============================] - 9s 138ms/step - loss: 0.0181 - accuracy: 0.9925 - val_loss: 1.3604 - val_accuracy: 0.7825
Epoch 11/100
68/68 [==============================] - 9s 140ms/step - loss: 0.0217 - accuracy: 0.9922 - val_loss: 1.1242 - val_accuracy: 0.7905
Epoch 12/100
68/68 [==============================] - 9s 138ms/step - loss: 0.0133 - accuracy: 0.9970 - val_loss: 1.2165 - val_accuracy: 0.8117
Epoch 13/100
68/68 [==============================] - 9s 139ms/step - loss: 0.0052 - accuracy: 0.9979 - val_loss: 1.9338 - val_accuracy: 0.7984
Epoch 1

Epoch 3/100
68/68 [==============================] - 9s 137ms/step - loss: 0.3143 - accuracy: 0.8673 - val_loss: 0.4836 - val_accuracy: 0.7825
Epoch 4/100
68/68 [==============================] - 9s 138ms/step - loss: 0.1475 - accuracy: 0.9500 - val_loss: 0.5810 - val_accuracy: 0.7825
Epoch 5/100
68/68 [==============================] - 9s 138ms/step - loss: 0.0761 - accuracy: 0.9734 - val_loss: 0.7086 - val_accuracy: 0.7745
Epoch 6/100
68/68 [==============================] - 9s 139ms/step - loss: 0.0632 - accuracy: 0.9797 - val_loss: 0.8209 - val_accuracy: 0.7798
Epoch 7/100
68/68 [==============================] - 9s 139ms/step - loss: 0.0303 - accuracy: 0.9894 - val_loss: 0.8803 - val_accuracy: 0.7851
Epoch 8/100
68/68 [==============================] - 9s 139ms/step - loss: 0.0206 - accuracy: 0.9931 - val_loss: 1.1781 - val_accuracy: 0.7639
Epoch 9/100
68/68 [==============================] - 9s 139ms/step - loss: 0.0151 - accuracy: 0.9962 - val_loss: 1.1622 - val_accuracy: 0.7772

...

Epoch 5/100
68/68 [==============================] - 9s 133ms/step - loss: 0.1558 - accuracy: 0.9422 - val_loss: 0.6256 - val_accuracy: 0.7666
Epoch 6/100
68/68 [==============================] - 9s 133ms/step - loss: 0.1059 - accuracy: 0.9691 - val_loss: 0.7428 - val_accuracy: 0.7507
Epoch 7/100
68/68 [==============================] - 9s 133ms/step - loss: 0.0806 - accuracy: 0.9774 - val_loss: 0.8156 - val_accuracy: 0.7798
Epoch 8/100
68/68 [==============================] - 9s 133ms/step - loss: 0.0472 - accuracy: 0.9910 - val_loss: 0.9501 - val_accuracy: 0.7480
Epoch 9/100
68/68 [==============================] - 9s 133ms/step - loss: 0.0604 - accuracy: 0.9804 - val_loss: 0.8050 - val_accuracy: 0.7719
Epoch 10/100
68/68 [==============================] - 9s 133ms/step - loss: 0.0413 - accuracy: 0.9888 - val_loss: 0.9520 - val_accuracy: 0.7480
Epoch 11/100
68/68 [==============================] - 9s 133ms/step - loss: 0.0258 - accuracy: 0.9923 - val_loss: 1.0240 - val_accuracy: 0.77

68/68 [==============================] - 9s 133ms/step - loss: 0.1334 - accuracy: 0.9527 - val_loss: 0.6014 - val_accuracy: 0.8148
Epoch 6/100
68/68 [==============================] - 9s 133ms/step - loss: 0.0917 - accuracy: 0.9686 - val_loss: 0.5501 - val_accuracy: 0.8042
Epoch 7/100
68/68 [==============================] - 9s 133ms/step - loss: 0.0594 - accuracy: 0.9841 - val_loss: 0.7349 - val_accuracy: 0.7698
Epoch 8/100
68/68 [==============================] - 9s 133ms/step - loss: 0.0527 - accuracy: 0.9809 - val_loss: 0.8154 - val_accuracy: 0.7884
Epoch 9/100
68/68 [==============================] - 9s 134ms/step - loss: 0.0488 - accuracy: 0.9849 - val_loss: 0.8442 - val_accuracy: 0.7884
Epoch 10/100
68/68 [==============================] - 9s 133ms/step - loss: 0.0363 - accuracy: 0.9890 - val_loss: 0.7277 - val_accuracy: 0.7804
Epoch 11/100
68/68 [==============================] - 9s 133ms/step - loss: 0.0539 - accuracy: 0.9787 - val_loss: 0.6952 - val_accuracy: 0.8069
Epoch 12/

Epoch 6/100
68/68 [==============================] - 9s 133ms/step - loss: 0.0891 - accuracy: 0.9712 - val_loss: 0.7422 - val_accuracy: 0.7566
Epoch 7/100
68/68 [==============================] - 9s 133ms/step - loss: 0.0472 - accuracy: 0.9849 - val_loss: 0.7147 - val_accuracy: 0.7778
Epoch 8/100
68/68 [==============================] - 9s 134ms/step - loss: 0.0418 - accuracy: 0.9890 - val_loss: 0.8930 - val_accuracy: 0.7407
Epoch 9/100
68/68 [==============================] - 9s 133ms/step - loss: 0.0474 - accuracy: 0.9888 - val_loss: 0.9418 - val_accuracy: 0.7751
Epoch 10/100
68/68 [==============================] - 9s 133ms/step - loss: 0.0262 - accuracy: 0.9905 - val_loss: 0.9131 - val_accuracy: 0.7884
Epoch 11/100
68/68 [==============================] - 9s 133ms/step - loss: 0.0196 - accuracy: 0.9918 - val_loss: 0.9803 - val_accuracy: 0.7646
Epoch 12/100
68/68 [==============================] - 9s 133ms/step - loss: 0.0290 - accuracy: 0.9908 - val_loss: 1.0252 - val_accuracy: 0.7

Epoch 1/100
68/68 [==============================] - 13s 128ms/step - loss: 0.6478 - accuracy: 0.6251 - val_loss: 0.5040 - val_accuracy: 0.7619
Epoch 2/100
68/68 [==============================] - 8s 116ms/step - loss: 0.3775 - accuracy: 0.8398 - val_loss: 0.4248 - val_accuracy: 0.7963
Epoch 3/100
68/68 [==============================] - 8s 117ms/step - loss: 0.2271 - accuracy: 0.9143 - val_loss: 0.5830 - val_accuracy: 0.7751
Epoch 4/100
68/68 [==============================] - 8s 117ms/step - loss: 0.1480 - accuracy: 0.9461 - val_loss: 0.6632 - val_accuracy: 0.7831
Epoch 5/100
68/68 [==============================] - 8s 119ms/step - loss: 0.0722 - accuracy: 0.9784 - val_loss: 0.7262 - val_accuracy: 0.7619
Epoch 6/100
68/68 [==============================] - 8s 119ms/step - loss: 0.0961 - accuracy: 0.9647 - val_loss: 0.8339 - val_accuracy: 0.7540
Epoch 7/100
68/68 [==============================] - 8s 118ms/step - loss: 0.0418 - accuracy: 0.9895 - val_loss: 0.9054 - val_accuracy: 0.764

Epoch 12/100
68/68 [==============================] - 9s 134ms/step - loss: 0.0315 - accuracy: 0.9915 - val_loss: 1.0167 - val_accuracy: 0.7698
Epoch 13/100
68/68 [==============================] - 9s 134ms/step - loss: 0.0156 - accuracy: 0.9962 - val_loss: 0.9476 - val_accuracy: 0.7672
Epoch 14/100
68/68 [==============================] - 9s 134ms/step - loss: 0.0264 - accuracy: 0.9928 - val_loss: 1.0295 - val_accuracy: 0.7593
Epoch 15/100
68/68 [==============================] - 9s 135ms/step - loss: 0.0096 - accuracy: 0.9988 - val_loss: 1.1639 - val_accuracy: 0.7725
Restoring model weights from the end of the best epoch.
Epoch 00015: early stopping
Test Accuracy: 79.10053133964539
Epoch 1/100
68/68 [==============================] - 14s 140ms/step - loss: 0.6548 - accuracy: 0.6164 - val_loss: 0.5037 - val_accuracy: 0.7619
Epoch 2/100
68/68 [==============================] - 9s 130ms/step - loss: 0.4409 - accuracy: 0.7984 - val_loss: 0.5079 - val_accuracy: 0.7804
Epoch 3/100
68/68 [=

## Summary

In [105]:
record.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
3,relu,4,82.010579,79.100531,76.190478,80.423278,80.158728,82.228118,83.819628,82.758623,81.432360,83.554375,81.167670
0,relu,1,83.068782,82.275134,81.216931,80.952382,80.687833,80.106103,80.371350,80.901855,80.371350,81.167108,81.111883
1,relu,2,79.629630,83.333331,82.539684,80.158728,79.365081,79.045093,79.045093,82.228118,81.167108,81.432360,80.794423
4,relu,5,83.068782,78.306878,82.010579,78.571427,80.423278,82.493371,81.697613,80.106103,80.901855,79.840851,80.742074
2,relu,3,80.687833,80.158728,80.687833,80.687833,83.068782,79.840851,81.962866,80.901855,79.045093,80.371350,80.741302
5,relu,6,81.216931,81.746030,74.867725,81.746030,80.158728,81.432360,83.023870,79.575598,79.840851,82.758623,80.636675
6,tanh,1,81.216931,78.571427,77.248675,80.423278,78.835976,81.167108,78.779840,80.371350,83.554375,79.310346,79.947931
8,tanh,3,81.746030,78.306878,77.777779,79.365081,81.746030,79.840851,79.045093,80.636603,80.901855,77.453583,79.681978
7,tanh,2,78.571427,78.571427,82.275134,81.746030,78.306878,76.127321,77.453583,77.453583,85.676390,79.310346,79.549212
11,tanh,6,79.629630,78.042328,80.423278,79.100531,80.423278,79.045093,75.596815,81.167108,81.697613,78.514588,79.364026


In [106]:
record[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

,AVG
Activation,
relu,81.167670
tanh,79.947931


In [107]:
report = record.sort_values(by='AVG', ascending=False)
report = report.to_excel('TCN_CR.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [60]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [61]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [62]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [63]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 5046 words present from 5336 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [140]:
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()

In [171]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index, emb_mean, emb_std):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution values
    embed_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

In [174]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i, emb_mean, emb_std)
em_matrix

array([[ 0.19468211,  0.08648376, -0.05924511, ..., -0.16683994,
        -0.09975549, -0.08595189],
       [-0.13509196, -0.07441947,  0.15388953, ..., -0.05400787,
        -0.13156594, -0.05996158],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## TCN Model

In [184]:
def define_model_2(kernel_size = 3, activation='relu', input_dim = None, 
                   output_dim=300, max_length = None, emb_matrix = None):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, 
                  output_dim=output_dim, 
                  input_length=max_length,
                  # Assign the embedding weight with word2vec embedding marix
                  weights = [emb_matrix],
                  # Set the weight to be not trainable (static)
                  trainable = False)(inp)
    
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [185]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_148"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_153 (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_165 (Embedding)       (None, 100, 300)     300000      input_153[0][0]                  
__________________________________________________________________________________________________
spatial_dropout1d_151 (SpatialD (None, 100, 300)     0           embedding_165[0][0]              
__________________________________________________________________________________________________
tnc1 (TCN)                      (None, 100, 128)     279936      spatial_dropout1d_151[0][0]      
__________________________________________________________________________________________

## Train and Test the Model

In [186]:
# class myCallback(tf.keras.callbacks.Callback):
#     # Overide the method on_epoch_end() for our benefit
#     def on_epoch_end(self, epoch, logs={}):
#         if (logs.get('accuracy') >= 0.9):
#             print("\nReached 90% accuracy so cancelling training!")
#             self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [192]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
print('Loading embedding statistics . . .')
emb_mean = emb_mean
emb_std = emb_std
print('Done!')
kernel_sizes = [1, 2, 3, 4, 5, 6, 7, 8]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)
exp = 0

for activation in activations:
    for kernel_size in kernel_sizes:
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_2(kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record2 = record2.append(temp, ignore_index=True)
        print()
        print(record2)
        print()

Loading embedding statistics . . .
Done!
-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------
Epoch 1/100
68/68 [==============================] - 16s 179ms/step - loss: 0.9523 - accuracy: 0.5849 - val_loss: 0.6465 - val_accuracy: 0.6481
Epoch 2/100
68/68 [==============================] - 12s 178ms/step - loss: 0.6539 - accuracy: 0.6386 - val_loss: 0.6262 - val_accuracy: 0.6508
Epoch 3/100
68/68 [==============================] - 12s 177ms/step - loss: 0.5935 - accuracy: 0.6575 - val_loss: 0.4295 - val_accuracy: 0.7989
Epoch 4/100
68/68 [==============================] - 12s 174ms/step - loss: 0.4496 - accuracy: 0.7930 - val_loss: 0.4063 - val_accuracy: 0.8175
Epoch 5/100
68/68 [==============================] - 10s 152ms/step - loss: 0.3483 - accuracy: 0.8452 - val_loss: 0.4377 - val_accuracy: 0.8307
Epoch 6/100
68/68 [==============================] - 8s 113ms/step - loss: 0.3061 - accuracy: 0.8773 - val_

Test Accuracy: 82.0105791091919
Epoch 1/100
68/68 [==============================] - 12s 117ms/step - loss: 0.7667 - accuracy: 0.5919 - val_loss: 0.6373 - val_accuracy: 0.6138
Epoch 2/100
68/68 [==============================] - 7s 109ms/step - loss: 0.5646 - accuracy: 0.6846 - val_loss: 0.4722 - val_accuracy: 0.7540
Epoch 3/100
68/68 [==============================] - 8s 125ms/step - loss: 0.4585 - accuracy: 0.7884 - val_loss: 0.4444 - val_accuracy: 0.7831
Epoch 4/100
68/68 [==============================] - 7s 110ms/step - loss: 0.3726 - accuracy: 0.8388 - val_loss: 0.4334 - val_accuracy: 0.7884
Epoch 5/100
68/68 [==============================] - 7s 110ms/step - loss: 0.3147 - accuracy: 0.8637 - val_loss: 0.4472 - val_accuracy: 0.8016
Epoch 6/100
68/68 [==============================] - 10s 152ms/step - loss: 0.2608 - accuracy: 0.8924 - val_loss: 0.5979 - val_accuracy: 0.7672
Epoch 7/100
68/68 [==============================] - 7s 110ms/step - loss: 0.2162 - accuracy: 0.9100 - val_l

68/68 [==============================] - 10s 155ms/step - loss: 0.1660 - accuracy: 0.9224 - val_loss: 0.4622 - val_accuracy: 0.8196
Epoch 14/100
68/68 [==============================] - 7s 110ms/step - loss: 0.1573 - accuracy: 0.9307 - val_loss: 0.8354 - val_accuracy: 0.7666
Epoch 15/100
68/68 [==============================] - 7s 110ms/step - loss: 0.1254 - accuracy: 0.9519 - val_loss: 0.7346 - val_accuracy: 0.7639
Epoch 16/100
68/68 [==============================] - 7s 110ms/step - loss: 0.1121 - accuracy: 0.9598 - val_loss: 0.7082 - val_accuracy: 0.8117
Epoch 17/100
68/68 [==============================] - 7s 110ms/step - loss: 0.0731 - accuracy: 0.9694 - val_loss: 0.7505 - val_accuracy: 0.8196
Epoch 18/100
68/68 [==============================] - 10s 150ms/step - loss: 0.0752 - accuracy: 0.9805 - val_loss: 0.6480 - val_accuracy: 0.8196
Epoch 19/100
68/68 [==============================] - 7s 110ms/step - loss: 0.0595 - accuracy: 0.9755 - val_loss: 0.6504 - val_accuracy: 0.8302
Epo

Epoch 17/100
68/68 [==============================] - 7s 110ms/step - loss: 0.0579 - accuracy: 0.9796 - val_loss: 0.9306 - val_accuracy: 0.7884
Epoch 18/100
68/68 [==============================] - 7s 110ms/step - loss: 0.0888 - accuracy: 0.9680 - val_loss: 0.6721 - val_accuracy: 0.7884
Epoch 19/100
68/68 [==============================] - 8s 112ms/step - loss: 0.0284 - accuracy: 0.9929 - val_loss: 1.0714 - val_accuracy: 0.7910
Epoch 20/100
68/68 [==============================] - 9s 135ms/step - loss: 0.0423 - accuracy: 0.9809 - val_loss: 1.1391 - val_accuracy: 0.8148
Epoch 21/100
68/68 [==============================] - 8s 111ms/step - loss: 0.0563 - accuracy: 0.9853 - val_loss: 1.1514 - val_accuracy: 0.7725
Epoch 22/100
68/68 [==============================] - 8s 111ms/step - loss: 0.0412 - accuracy: 0.9879 - val_loss: 0.9901 - val_accuracy: 0.8016
Epoch 23/100
68/68 [==============================] - 8s 111ms/step - loss: 0.0291 - accuracy: 0.9909 - val_loss: 0.9242 - val_accuracy:

68/68 [==============================] - 7s 110ms/step - loss: 0.0229 - accuracy: 0.9928 - val_loss: 0.6059 - val_accuracy: 0.8382
Epoch 23/100
68/68 [==============================] - 8s 111ms/step - loss: 0.0455 - accuracy: 0.9859 - val_loss: 0.8391 - val_accuracy: 0.8223
Epoch 24/100
68/68 [==============================] - 7s 110ms/step - loss: 0.0235 - accuracy: 0.9924 - val_loss: 0.9137 - val_accuracy: 0.8382
Epoch 25/100
68/68 [==============================] - 7s 110ms/step - loss: 0.0287 - accuracy: 0.9917 - val_loss: 0.8979 - val_accuracy: 0.8276
Restoring model weights from the end of the best epoch.
Epoch 00025: early stopping
Test Accuracy: 84.08488035202026
Epoch 1/100
68/68 [==============================] - 11s 104ms/step - loss: 0.7617 - accuracy: 0.5955 - val_loss: 0.6360 - val_accuracy: 0.6340
Epoch 2/100
68/68 [==============================] - 6s 94ms/step - loss: 0.6189 - accuracy: 0.6502 - val_loss: 0.5004 - val_accuracy: 0.7533
Epoch 3/100
68/68 [===============

...

68/68 [==============================] - 8s 112ms/step - loss: 0.0533 - accuracy: 0.9836 - val_loss: 0.9825 - val_accuracy: 0.8360
Epoch 21/100
68/68 [==============================] - 11s 156ms/step - loss: 0.0432 - accuracy: 0.9824 - val_loss: 1.0844 - val_accuracy: 0.8069
Epoch 22/100
68/68 [==============================] - 10s 151ms/step - loss: 0.0726 - accuracy: 0.9701 - val_loss: 0.9826 - val_accuracy: 0.8254
Epoch 23/100
68/68 [==============================] - 8s 114ms/step - loss: 0.0328 - accuracy: 0.9868 - val_loss: 0.6884 - val_accuracy: 0.8333
Epoch 24/100
68/68 [==============================] - 8s 114ms/step - loss: 0.0727 - accuracy: 0.9743 - val_loss: 1.1288 - val_accuracy: 0.8201
Epoch 25/100
68/68 [==============================] - 11s 162ms/step - loss: 0.0321 - accuracy: 0.9874 - val_loss: 1.1193 - val_accuracy: 0.8254
Epoch 26/100
68/68 [==============================] - 8s 112ms/step - loss: 0.0299 - accuracy: 0.9884 - val_loss: 1.1945 - val_accuracy: 0.7884
Ep

68/68 [==============================] - 7s 110ms/step - loss: 0.0882 - accuracy: 0.9717 - val_loss: 0.4786 - val_accuracy: 0.8488
Epoch 12/100
68/68 [==============================] - 7s 110ms/step - loss: 0.0850 - accuracy: 0.9709 - val_loss: 0.6525 - val_accuracy: 0.8249
Epoch 13/100
68/68 [==============================] - 8s 113ms/step - loss: 0.0768 - accuracy: 0.9719 - val_loss: 0.4953 - val_accuracy: 0.8515
Epoch 14/100
68/68 [==============================] - 11s 163ms/step - loss: 0.0677 - accuracy: 0.9770 - val_loss: 0.7663 - val_accuracy: 0.8249
Epoch 15/100
68/68 [==============================] - 8s 117ms/step - loss: 0.0498 - accuracy: 0.9837 - val_loss: 0.5628 - val_accuracy: 0.8408
Restoring model weights from the end of the best epoch.
Epoch 00015: early stopping
Test Accuracy: 85.67638993263245
Epoch 1/100
68/68 [==============================] - 13s 120ms/step - loss: 0.7964 - accuracy: 0.5880 - val_loss: 0.6547 - val_accuracy: 0.6790
Epoch 2/100
68/68 [============

Epoch 1/100
68/68 [==============================] - 12s 118ms/step - loss: 0.7746 - accuracy: 0.6003 - val_loss: 0.6405 - val_accuracy: 0.6481
Epoch 2/100
68/68 [==============================] - 7s 109ms/step - loss: 0.6242 - accuracy: 0.6377 - val_loss: 0.5137 - val_accuracy: 0.6878
Epoch 3/100
68/68 [==============================] - 7s 110ms/step - loss: 0.5616 - accuracy: 0.6785 - val_loss: 0.4655 - val_accuracy: 0.7857
Epoch 4/100
68/68 [==============================] - 7s 110ms/step - loss: 0.4426 - accuracy: 0.8025 - val_loss: 0.4243 - val_accuracy: 0.7963
Epoch 5/100
68/68 [==============================] - 7s 110ms/step - loss: 0.3777 - accuracy: 0.8255 - val_loss: 0.3926 - val_accuracy: 0.8016
Epoch 6/100
68/68 [==============================] - 8s 111ms/step - loss: 0.3110 - accuracy: 0.8772 - val_loss: 0.3884 - val_accuracy: 0.8280
Epoch 7/100
68/68 [==============================] - 8s 111ms/step - loss: 0.2541 - accuracy: 0.9070 - val_loss: 0.4064 - val_accuracy: 0.809

68/68 [==============================] - 8s 113ms/step - loss: 0.0950 - accuracy: 0.9682 - val_loss: 0.6381 - val_accuracy: 0.8016
Epoch 15/100
68/68 [==============================] - 9s 126ms/step - loss: 0.0988 - accuracy: 0.9637 - val_loss: 0.7333 - val_accuracy: 0.7963
Epoch 16/100
68/68 [==============================] - 8s 112ms/step - loss: 0.0733 - accuracy: 0.9771 - val_loss: 0.6848 - val_accuracy: 0.8280
Epoch 17/100
68/68 [==============================] - 8s 112ms/step - loss: 0.0377 - accuracy: 0.9865 - val_loss: 0.7033 - val_accuracy: 0.8333
Epoch 18/100
68/68 [==============================] - 9s 137ms/step - loss: 0.0605 - accuracy: 0.9807 - val_loss: 0.6877 - val_accuracy: 0.8333
Epoch 19/100
68/68 [==============================] - 8s 112ms/step - loss: 0.0555 - accuracy: 0.9815 - val_loss: 0.8431 - val_accuracy: 0.8016
Epoch 20/100
68/68 [==============================] - 10s 150ms/step - loss: 0.0641 - accuracy: 0.9794 - val_loss: 0.7053 - val_accuracy: 0.8333
Epoc

68/68 [==============================] - 8s 112ms/step - loss: 0.0213 - accuracy: 0.9941 - val_loss: 0.8909 - val_accuracy: 0.7719
Restoring model weights from the end of the best epoch.
Epoch 00019: early stopping
Test Accuracy: 83.02386999130249
Epoch 1/100
68/68 [==============================] - 13s 121ms/step - loss: 0.7357 - accuracy: 0.6160 - val_loss: 0.6442 - val_accuracy: 0.6472
Epoch 2/100
68/68 [==============================] - 8s 111ms/step - loss: 0.6234 - accuracy: 0.6555 - val_loss: 0.5052 - val_accuracy: 0.7533
Epoch 3/100
68/68 [==============================] - 10s 146ms/step - loss: 0.4939 - accuracy: 0.7509 - val_loss: 0.4407 - val_accuracy: 0.8064
Epoch 4/100
68/68 [==============================] - 8s 112ms/step - loss: 0.3768 - accuracy: 0.8261 - val_loss: 0.4070 - val_accuracy: 0.8011
Epoch 5/100
68/68 [==============================] - 10s 153ms/step - loss: 0.3449 - accuracy: 0.8482 - val_loss: 0.3873 - val_accuracy: 0.8276
Epoch 6/100
68/68 [===============

## Summary

In [193]:
record2.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
1,relu,2,87.301588,82.275134,84.391534,84.920633,80.158728,84.084880,82.493371,81.167108,86.206895,85.676390,83.867626
2,relu,3,83.597887,85.449737,84.126985,80.952382,82.010579,81.697613,85.941643,86.472148,84.350133,83.289123,83.788823
4,relu,5,81.216931,82.010579,85.449737,82.539684,83.862436,85.676390,85.941643,81.432360,83.819628,85.676390,83.762578
7,relu,8,83.068782,83.068782,83.862436,83.333331,84.391534,85.941643,83.289123,81.962866,83.023870,84.084880,83.602725
3,relu,4,82.539684,81.746030,84.656084,82.539684,85.714287,81.962866,84.350133,82.758623,83.819628,85.676390,83.576341
6,relu,7,81.481481,85.185188,83.597887,84.656084,84.126985,79.310346,84.350133,81.962866,85.676390,84.615386,83.496274
5,relu,6,82.804233,83.597887,85.449737,83.068782,81.481481,84.350133,81.432360,86.206895,81.697613,84.350133,83.443925
0,relu,1,84.126985,84.391534,82.010579,80.158728,87.566137,83.819628,80.371350,83.819628,84.350133,82.758623,83.337333


In [194]:
record2[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

,AVG
Activation,
relu,83.867626


In [195]:
report = record2.sort_values(by='AVG', ascending=False)
report = report.to_excel('TCN_CR_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## CNN Model

In [196]:
def define_model_3(kernel_size = 3, activation='relu', input_dim = None, 
                   output_dim=300, max_length = None, emb_matrix = None):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, 
                  output_dim=output_dim, 
                  input_length=max_length,
                  # Assign the embedding weight with word2vec embedding marix
                  weights = [emb_matrix],
                  # Set the weight to be not trainable (static)
                  trainable = True)(inp)
    
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [197]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_232"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_237 (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_249 (Embedding)       (None, 100, 300)     300000      input_237[0][0]                  
__________________________________________________________________________________________________
spatial_dropout1d_235 (SpatialD (None, 100, 300)     0           embedding_249[0][0]              
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     279936      spatial_dropout1d_235[0][0]      
__________________________________________________________________________________________

## Train and Test the Model

In [198]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=10, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [199]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
print('Loading embedding statistics . . .')
emb_mean = emb_mean
emb_std = emb_std
print('Done!')
kernel_sizes = [1, 2, 3, 4, 5, 6, 7, 8]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp = 0

for activation in activations:
    for kernel_size in kernel_sizes:
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_3(kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model
            model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                      callbacks=[callbacks], validation_data=(Xtest, test_y))

            # evaluate the model
            loss, acc = model.evaluate(Xtest, test_y, verbose=0)
            print('Test Accuracy: {}'.format(acc*100))

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record3 = record3.append(temp, ignore_index=True)
        print()
        print(record3)
        print()

Loading embedding statistics . . .
Done!
-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------


C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/100
68/68 [==============================] - 19s 174ms/step - loss: 0.6995 - accuracy: 0.6087 - val_loss: 0.6701 - val_accuracy: 0.6058
Epoch 2/100
68/68 [==============================] - 10s 151ms/step - loss: 0.6097 - accuracy: 0.6594 - val_loss: 0.4716 - val_accuracy: 0.7540
Epoch 3/100
68/68 [==============================] - 13s 189ms/step - loss: 0.3725 - accuracy: 0.8211 - val_loss: 0.4076 - val_accuracy: 0.7989
Epoch 4/100
68/68 [==============================] - 11s 161ms/step - loss: 0.2503 - accuracy: 0.9031 - val_loss: 0.4231 - val_accuracy: 0.7937
Epoch 5/100
68/68 [==============================] - 12s 171ms/step - loss: 0.1588 - accuracy: 0.9371 - val_loss: 0.4244 - val_accuracy: 0.8148
Epoch 6/100
68/68 [==============================] - 10s 142ms/step - loss: 0.1151 - accuracy: 0.9657 - val_loss: 0.6858 - val_accuracy: 0.7804
Epoch 7/100
68/68 [==============================] - 12s 174ms/step - loss: 0.1113 - accuracy: 0.9631 - val_loss: 0.6148 - val_accuracy:

Epoch 12/100
68/68 [==============================] - 9s 140ms/step - loss: 0.0381 - accuracy: 0.9843 - val_loss: 0.7009 - val_accuracy: 0.8170
Epoch 13/100
68/68 [==============================] - 9s 139ms/step - loss: 0.0233 - accuracy: 0.9946 - val_loss: 0.9089 - val_accuracy: 0.8249
Epoch 14/100
68/68 [==============================] - 10s 141ms/step - loss: 0.0090 - accuracy: 0.9967 - val_loss: 1.3463 - val_accuracy: 0.8090
Epoch 15/100
68/68 [==============================] - 9s 139ms/step - loss: 0.0319 - accuracy: 0.9918 - val_loss: 0.7912 - val_accuracy: 0.8090
Restoring model weights from the end of the best epoch.
Epoch 00015: early stopping
Test Accuracy: 84.3501329421997
Epoch 1/100
68/68 [==============================] - 14s 148ms/step - loss: 0.6817 - accuracy: 0.6424 - val_loss: 0.6240 - val_accuracy: 0.6419
Epoch 2/100
68/68 [==============================] - 9s 139ms/step - loss: 0.5567 - accuracy: 0.6951 - val_loss: 0.4694 - val_accuracy: 0.7958
Epoch 3/100
68/68 [=

68/68 [==============================] - 9s 139ms/step - loss: 0.0272 - accuracy: 0.9911 - val_loss: 0.9965 - val_accuracy: 0.7937
Epoch 17/100
68/68 [==============================] - 9s 139ms/step - loss: 0.0284 - accuracy: 0.9899 - val_loss: 1.1745 - val_accuracy: 0.8016
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Test Accuracy: 83.8624358177185
Epoch 1/100
68/68 [==============================] - 14s 145ms/step - loss: 0.7331 - accuracy: 0.6004 - val_loss: 0.6295 - val_accuracy: 0.6270
Epoch 2/100
68/68 [==============================] - 9s 136ms/step - loss: 0.5223 - accuracy: 0.7027 - val_loss: 0.5286 - val_accuracy: 0.7698
Epoch 3/100
68/68 [==============================] - 9s 138ms/step - loss: 0.3284 - accuracy: 0.8595 - val_loss: 0.4370 - val_accuracy: 0.7963
Epoch 4/100
68/68 [==============================] - 9s 139ms/step - loss: 0.1885 - accuracy: 0.9284 - val_loss: 0.4975 - val_accuracy: 0.7831
Epoch 5/100
68/68 [=================

...

68/68 [==============================] - 33s 485ms/step - loss: 0.0176 - accuracy: 0.9907 - val_loss: 2.0697 - val_accuracy: 0.8117
Epoch 22/100
68/68 [==============================] - 34s 500ms/step - loss: 0.0277 - accuracy: 0.9875 - val_loss: 1.1840 - val_accuracy: 0.8196
Epoch 23/100
68/68 [==============================] - 31s 458ms/step - loss: 0.0187 - accuracy: 0.9906 - val_loss: 1.8752 - val_accuracy: 0.8064
Epoch 24/100
68/68 [==============================] - 26s 388ms/step - loss: 0.0121 - accuracy: 0.9909 - val_loss: 2.0272 - val_accuracy: 0.8170
Epoch 25/100
68/68 [==============================] - 23s 339ms/step - loss: 0.0095 - accuracy: 0.9938 - val_loss: 2.5417 - val_accuracy: 0.8170
Epoch 26/100
68/68 [==============================] - 32s 475ms/step - loss: 0.0103 - accuracy: 0.9923 - val_loss: 2.3221 - val_accuracy: 0.7984
Epoch 27/100
68/68 [==============================] - 25s 363ms/step - loss: 0.0179 - accuracy: 0.9898 - val_loss: 2.0600 - val_accuracy: 0.801

Epoch 5/100
68/68 [==============================] - 9s 138ms/step - loss: 0.1834 - accuracy: 0.9329 - val_loss: 0.4090 - val_accuracy: 0.8355
Epoch 6/100
68/68 [==============================] - 9s 138ms/step - loss: 0.1301 - accuracy: 0.9522 - val_loss: 0.5652 - val_accuracy: 0.8196
Epoch 7/100
68/68 [==============================] - 9s 138ms/step - loss: 0.0918 - accuracy: 0.9686 - val_loss: 0.6084 - val_accuracy: 0.8223
Epoch 8/100
68/68 [==============================] - 9s 137ms/step - loss: 0.0793 - accuracy: 0.9731 - val_loss: 0.7765 - val_accuracy: 0.8302
Epoch 9/100
68/68 [==============================] - 9s 138ms/step - loss: 0.0578 - accuracy: 0.9848 - val_loss: 0.7334 - val_accuracy: 0.8196
Epoch 10/100
68/68 [==============================] - 9s 138ms/step - loss: 0.0456 - accuracy: 0.9892 - val_loss: 0.8815 - val_accuracy: 0.8064
Epoch 11/100
68/68 [==============================] - 9s 139ms/step - loss: 0.0634 - accuracy: 0.9864 - val_loss: 0.7236 - val_accuracy: 0.80

Epoch 12/100
68/68 [==============================] - 10s 140ms/step - loss: 0.0184 - accuracy: 0.9957 - val_loss: 0.8809 - val_accuracy: 0.7937
Epoch 13/100
68/68 [==============================] - 10s 141ms/step - loss: 0.0323 - accuracy: 0.9902 - val_loss: 1.0183 - val_accuracy: 0.7857
Epoch 14/100
68/68 [==============================] - 10s 140ms/step - loss: 0.0342 - accuracy: 0.9887 - val_loss: 1.0361 - val_accuracy: 0.7593
Epoch 15/100
68/68 [==============================] - 10s 140ms/step - loss: 0.0220 - accuracy: 0.9929 - val_loss: 1.2376 - val_accuracy: 0.7831
Epoch 16/100
68/68 [==============================] - 10s 140ms/step - loss: 0.0097 - accuracy: 0.9960 - val_loss: 1.2267 - val_accuracy: 0.7778
Epoch 17/100
68/68 [==============================] - 10s 140ms/step - loss: 0.0251 - accuracy: 0.9913 - val_loss: 1.2235 - val_accuracy: 0.7910
Restoring model weights from the end of the best epoch.
Epoch 00017: early stopping
Test Accuracy: 79.89417910575867
Epoch 1/100
6

68/68 [==============================] - 10s 144ms/step - loss: 0.0123 - accuracy: 0.9965 - val_loss: 1.2089 - val_accuracy: 0.7984
Restoring model weights from the end of the best epoch.
Epoch 00016: early stopping
Test Accuracy: 82.49337077140808
Epoch 1/100
68/68 [==============================] - 15s 152ms/step - loss: 0.7719 - accuracy: 0.5961 - val_loss: 0.6769 - val_accuracy: 0.5889
Epoch 2/100
68/68 [==============================] - 10s 142ms/step - loss: 0.6495 - accuracy: 0.6564 - val_loss: 0.6337 - val_accuracy: 0.5889
Epoch 3/100
68/68 [==============================] - 10s 143ms/step - loss: 0.5441 - accuracy: 0.6894 - val_loss: 0.5537 - val_accuracy: 0.7029
Epoch 4/100
68/68 [==============================] - 10s 144ms/step - loss: 0.3611 - accuracy: 0.8502 - val_loss: 0.4537 - val_accuracy: 0.8011
Epoch 5/100
68/68 [==============================] - 10s 143ms/step - loss: 0.2211 - accuracy: 0.9148 - val_loss: 0.4608 - val_accuracy: 0.8011
Epoch 6/100
68/68 [============

68/68 [==============================] - 10s 145ms/step - loss: 0.0477 - accuracy: 0.9816 - val_loss: 0.7279 - val_accuracy: 0.7937
Epoch 9/100
68/68 [==============================] - 13s 189ms/step - loss: 0.0390 - accuracy: 0.9880 - val_loss: 0.8541 - val_accuracy: 0.7831
Epoch 10/100
68/68 [==============================] - 10s 145ms/step - loss: 0.0202 - accuracy: 0.9948 - val_loss: 0.9186 - val_accuracy: 0.7698
Epoch 11/100
68/68 [==============================] - 10s 145ms/step - loss: 0.0249 - accuracy: 0.9905 - val_loss: 0.9564 - val_accuracy: 0.7937
Epoch 12/100
68/68 [==============================] - 12s 181ms/step - loss: 0.0212 - accuracy: 0.9940 - val_loss: 0.7240 - val_accuracy: 0.7963
Epoch 13/100
68/68 [==============================] - 10s 144ms/step - loss: 0.0177 - accuracy: 0.9955 - val_loss: 1.0828 - val_accuracy: 0.7857
Epoch 14/100
68/68 [==============================] - 10s 145ms/step - loss: 0.0371 - accuracy: 0.9853 - val_loss: 0.9580 - val_accuracy: 0.7937

## Summary

In [200]:
record3.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,relu,1,83.068782,84.126985,81.216931,81.746030,82.275134,84.880638,84.350133,82.758623,83.289123,81.432360,82.914474
4,relu,5,81.216931,83.862436,84.391534,83.333331,84.126985,83.023870,81.167108,81.432360,83.023870,83.289123,82.886755
3,relu,4,82.539684,81.216931,82.804233,82.010579,83.333331,80.901855,83.554375,84.350133,81.697613,85.411143,82.781988
1,relu,2,81.746030,82.010579,83.862436,81.216931,82.275134,81.962866,86.737400,83.023870,84.350133,80.371350,82.755673
5,relu,6,80.952382,82.804233,83.597887,81.746030,83.597887,84.880638,83.554375,81.697613,81.697613,80.636603,82.516526
2,relu,3,79.365081,81.216931,81.481481,81.216931,85.185188,83.819628,81.697613,84.880638,83.023870,83.023870,82.491123
6,relu,7,82.275134,79.894179,82.275134,81.216931,82.804233,83.819628,84.084880,82.493371,83.289123,82.228118,82.438073
7,relu,8,81.481481,84.656084,85.978836,80.952382,81.746030,85.941643,83.023870,81.697613,67.904508,81.962866,81.534531


In [201]:
report = record3.sort_values(by='AVG', ascending=False)
report = report.to_excel('TCN_CR_3.xlsx', sheet_name='dynamic')